### Setting Up the Data Warehouse

Installations required:
- [MySQL](https://dev.mysql.com/downloads/installer/)
- mysql-connector-python (pip)

In [1]:
# Database configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234567',
    'port': '3306',  # Default MySQL port
    'database': 'Group4MCO1'  # Replace with your database name
}

In [2]:
from SteamDB import SteamDB

db = SteamDB(db_config=db_config)
db.create_steam_db()

CSV loaded into DataFrame.
Connected to MySQL server.
Connected to database 'Group4MCO1'.
SQL query executed successfully!


In [3]:
db.populate_steam_db()
db.execute_sql("SELECT * FROM dim_game LIMIT 5")

An error occurred: (mysql.connector.errors.OperationalError) 2013 (HY000): Lost connection to MySQL server during query
[SQL: INSERT INTO dim_game (`gameID`, name, `releaseDate`, `requiredAge`, `aboutTheGame`, `websiteURL`, `supportURL`, `supportEmail`, `supportedLanguages`, `fullAudioLanguages`, `headerImageHREF`, categories, tags, genres) VALUES (%(gameID)s, %(name)s, %(releaseDate)s, %(requiredAge)s, %(aboutTheGame)s, %(websiteURL)s, %(supportURL)s, %(supportEmail)s, %(supportedLanguages)s, %(fullAudioLanguages)s, %(headerImageHREF)s, %(categories)s, %(tags)s, %(genres)s)]
[parameters: [{'gameID': 20200, 'name': 'Galactic Bowling', 'releaseDate': '10-21-2008', 'requiredAge': 0, 'aboutTheGame': 'Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player c ... (745 characters truncated) ... tiplayer play Modes. Super Powers, Special Balls, and Whammies. Unlockable Characters, Environments, and Min